In [6]:
%cd '/content/drive/MyDrive/Colab Notebooks/kaggle/house_pricing'

/content/drive/MyDrive/Colab Notebooks/kaggle/house_pricing


In [7]:
from common import *
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.compose import TransformedTargetRegressor

In [8]:
x_train, y_train, x_test = read_preprocessed_data(postfix='preprocessed')
print(x_train.shape, y_train.shape, x_test.shape)

(1451, 309) (1451, 1) (1459, 309)


In [9]:
def create_transformed_regressor(regressor):
  return TransformedTargetRegressor(
      regressor=regressor, func=np.log1p, inverse_func=np.expm1)

# Lasso

In [30]:
%%time
params = {'regressor__alpha': np.arange(0.5, 10)}
# model = Lasso(max_iter=100000)
model = TransformedTargetRegressor(regressor=Lasso(alpha=0), func=np.log1p, 
    inverse_func=np.expm1)
# print(tt.get_params().keys())
# model = TransformedTargetRegressor(regressor=Lasso(alpha=0), 
#                                    func=np.log1p, inverse_func=np.expm1)
print(grid_search(x_train, y_train, model, params))

(-20912.095477611518, {'regressor__alpha': 0.5})
CPU times: user 5.43 s, sys: 5.02 s, total: 10.5 s
Wall time: 5.42 s


In [34]:
model = Lasso(alpha=201, max_iter=10000)
print(get_cv_score(x_train, y_train, model))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.549e+09, tolerance: 7.514e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.348e+09, tolerance: 7.228e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisa

16163.372056041237


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.078e+09, tolerance: 7.158e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


In [ ]:
from scipy.optimize import minimize_scalar
def fun(alpha, x, y):
    model = make_pipeline(RobustScaler(), Lasso(alpha=alpha))
    score =  get_score(x, y, model)
    print('alpha', alpha, 'score', score)
    return score

res = minimize_scalar(fun, (0), args=(x, y), bounds=[0, 250], method='bounded')
print(res)

In [ ]:
model = create_transformed_regressor(Lasso(alpha=0))
print(get_score(x, y, model))

In [ ]:
y_log = np.log1p(y)
model = Lasso(alpha = 0)
print(get_score(x, y_log, model, log_transform=True))
print(get_score(x, y_log, model))

# Ridge

In [ ]:
ridge = Ridge(alpha=24.9)
score = get_score(x, y, ridge)
scores['ridge'] = score

In [ ]:
params = {'alpha': np.arange(10, 30)}
model = Ridge()
gs = GridSearchCV(model, params, cv=5, scoring='neg_mean_absolute_error')
gs.fit(x, y)
print(-gs.best_score_, gs.best_params_)

In [ ]:
params = {'regressor__alpha': np.arange(0, 200)}
model = TransformedTargetRegressor(regressor=Ridge(), 
                                   func=np.log1p, inverse_func=np.expm1)
gs = GridSearchCV(model, params, cv=5, scoring='neg_mean_absolute_error')
gs.fit(x, y)
print(-gs.best_score_, gs.best_params_)